In [ ]:
strategy_name = "IC"
verbose = True
plot_graph = False
recent_data_only=True

In [ ]:
%load_ext autoreload
%autoreload 2

# equity_index_future_basis_analysis
股指期货基差套利分析

## 背景原理

### 关于股指期货

[https://wiki.mbalib.com/wiki/%E8%82%A1%E6%8C%87%E6%9C%9F%E8%B4%A7](https://wiki.mbalib.com/wiki/%E8%82%A1%E6%8C%87%E6%9C%9F%E8%B4%A7)

股指期货相当于对股票指数中所包含的一篮子股票打包作为期货进行买卖的概念。买入

### 关于贴水和升水

股指期货与现货指数价格的差被称为基差，当股指期货价格高于现货指数价格时，股指期货处于升水，基差为正;反之，股指期货处于贴水，基差为负。

### 关于股指期货贴水获利的原理

[https://xueqiu.com/1445029417/143207882](https://xueqiu.com/1445029417/143207882)



## 回测分析

回测的目的是：
1. 确认在过去的回测期间内，策略是有效可盈利的。
2. 在策略的大框架下选择标的证券。

### 安装环境依赖

### 策略1

最基础的策略为，选定一个标的合约，并不断展期持有。例如持有当月合约，在到期时展期至下一时间的合约。例如在4月份持有 IC2104，并在4月交割日展期为 IC2105。

在这个策略中可选参数为：
1. 合约的期限。当月、下月、当季、下季。
2. 标的。IC、IF、IH

回测后发现，持有下月的 IC 合约，基差获利最多。

### 策略2

在策略1的基础上，每次展期所获得的基差收益来源于两个合约的基差之差。而基差之差是会浮动变化的，具体表现为：

在上涨预期强或者上涨趋势中，基差缩小，甚至部分时候会出现升水。
在震荡预期中，基差保持不变。
在下跌预期中，基差扩大。

利用这个特性，可以对策略1进行优化：
1. 在过去N个交易日中，如果是上涨趋势，则暂缓展期，回退至持有当月合约的策略。
2. 在过去N个交易日中，如果是震荡预期或下跌预期，则持有次月合约，次月合约转当月合约时进行展期。

展期本质上是走某个时间点，将时间价值兑换为对应的基差价值。而对应时点的基差价值是由当时的做空成本决定的，择时展期本质上就是对做空成本的择时。

具体策略表现为，计算每个合约的年化基差率，并将年化基差率作为时间序列进行计算，同时生成技术指标布林通道，布林通道参数为 120 日均值，0.5 标准差。当目前的年化基差率大于布林通道上轨时进行展期。2017年至今回测可以提高20%收益率。

In [ ]:
from spread_rolling import SpreadRollingStrategyBackTestingWrapper
from market_data.data_definition import ic_daily_tick_data, ic_index_data
from datetime import datetime

In [ ]:
s1 = SpreadRollingStrategyBackTestingWrapper(future_data=ic_daily_tick_data, underlying_data=ic_index_data)


In [ ]:
s1.download_data()
s1.back_test_and_get_today_target_pos(start_date=datetime(2019,1,1))